In [ ]:
"""
SIMPLE RANGE DISPLAY - TODAY'S 10:00-11:15 AM RANGE
Just shows the high and low for today's trading range
"""
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, time
import pytz

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"
CONTRACT_ID = "CON.F.US.MNQ.H26"

# Range parameters
RANGE_START_HOUR = 10
RANGE_START_MINUTE = 0
RANGE_END_HOUR = 11
RANGE_END_MINUTE = 15

def authenticate():
    resp = requests.post(
        f"{API_BASE}/Auth/loginKey",
        json={"userName": USERNAME, "apiKey": KEY},
        headers={"accept": "text/plain", "Content-Type": "application/json"},
        timeout=15
    )
    return {
        "Authorization": f"Bearer {resp.json()['token']}",
        "accept": "text/plain",
        "Content-Type": "application/json"
    }

def get_todays_range(headers):
    """Get today's 10:00-11:15 AM range"""
    ny_tz = pytz.timezone('America/New_York')
    now = datetime.now(ny_tz)
    
    # Get bars from 9:30 AM to now
    start_time = now.replace(hour=9, minute=30, second=0, microsecond=0)
    end_time = now
    
    payload = {
        "contractId": CONTRACT_ID,
        "live": False,  # Changed to False - some APIs don't support live=True
        "startTime": start_time.astimezone(pytz.UTC).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "endTime": end_time.astimezone(pytz.UTC).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "unit": 2,
        "unitNumber": 5,
        "limit": 100,
        "includePartialBar": False
    }
    
    resp = requests.post(
        f"{API_BASE}/History/retrieveBars",
        json=payload,
        headers=headers,
        timeout=30
    )
    
    if resp.status_code != 200:
        print(f"❌ API Error: {resp.status_code}")
        return None, None, []
    
    response_data = resp.json()
    bars = response_data.get("bars")
    
    if bars is None or not bars:
        print(f"❌ No bars returned from API")
        print(f"Response: {response_data}")
        return None, None, []
    
    # Filter bars within 10:00-11:15 range period
    range_bars = []
    for bar in bars:
        timestamp = bar['t'].replace('+00:00', 'Z')
        dt_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        dt_utc = pytz.utc.localize(dt_utc)
        dt_ny = dt_utc.astimezone(ny_tz)
        
        bar_time = dt_ny.time()
        range_start = time(RANGE_START_HOUR, RANGE_START_MINUTE)
        range_end = time(RANGE_END_HOUR, RANGE_END_MINUTE)
        
        if range_start <= bar_time < range_end:
            range_bars.append({
                'time': dt_ny.strftime('%I:%M %p'),
                'high': bar['h'],
                'low': bar['l']
            })
    
    if not range_bars:
        return None, None, []
    
    high = max(bar['high'] for bar in range_bars)
    low = min(bar['low'] for bar in range_bars)
    
    return high, low, range_bars

def main():
    print("=" * 70)
    print("TODAY'S RANGE - 10:00-11:15 AM ET")
    print("=" * 70)
    
    ny_tz = pytz.timezone('America/New_York')
    now = datetime.now(ny_tz)
    
    print(f"\n📅 Date: {now.strftime('%A, %B %d, %Y')}")
    print(f"⏰ Current Time: {now.strftime('%I:%M:%S %p')} ET")
    
    print("\n[1/2] Authenticating...")
    headers = authenticate()
    print("✅ Authenticated")
    
    print("\n[2/2] Fetching range data...")
    range_high, range_low, bars = get_todays_range(headers)
    
    if range_high is None:
        print("❌ No range data available yet (market may not have opened)")
        return
    
    range_size = range_high - range_low
    
    print("✅ Range calculated")
    
    print("\n" + "=" * 70)
    print("📊 TODAY'S TRADING RANGE")
    print("=" * 70)
    
    print(f"\n🔴 RANGE HIGH:  {range_high:.2f}")
    print(f"🟢 RANGE LOW:   {range_low:.2f}")
    print(f"📏 RANGE SIZE:  {range_size:.2f} points")
    
    print(f"\n📈 TRADING RULES:")
    print(f"   • LONG Setup:  Price breaks BELOW {range_low:.2f}, then closes back INSIDE range")
    print(f"   • SHORT Setup: Price breaks ABOVE {range_high:.2f}, then closes back INSIDE range")
    
    print(f"\n⚙️  POSITION SIZING:")
    print(f"   • Contracts: 20")
    print(f"   • Stop Loss: 10 points")
    print(f"   • Take Profit: 100 points")
    
    current_time = now.time()
    range_complete = time(11, 15)
    
    if current_time < range_complete:
        time_left = datetime.combine(now.date(), range_complete) - datetime.combine(now.date(), current_time)
        minutes_left = time_left.seconds // 60
        print(f"\n⏳ Range still forming... {minutes_left} minutes until 11:15 AM ET")
    else:
        print(f"\n✅ Range complete - Ready to trade!")
    
    print("\n" + "=" * 70)
    print("RANGE BREAKDOWN (5-min bars)")
    print("=" * 70)
    print(f"\n{'Time':<12}{'High':<12}{'Low':<12}")
    print("-" * 70)
    
    for bar in bars:
        print(f"{bar['time']:<12}{bar['high']:<12.2f}{bar['low']:<12.2f}")
    
    print("\n" + "=" * 70)
    print("✅ DONE")
    print("=" * 70)

if __name__ == "__main__":
    main()